# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

student_data.head()




,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [54]:
total_schools = len(school_data_complete["school_name"].unique())
total_students = len(school_data_complete["student_name"])
total_budget = school_data["budget"].sum()

average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
overall_passing = (average_reading_score + average_math_score)/2

passing_math= school_data_complete.loc[school_data_complete["math_score"]>= 70]
passing_reading= school_data_complete.loc[school_data_complete["reading_score"]>= 70]

percentage_passing_math = len(passing_math)/total_students*100
percentage_passing_reading= len(passing_reading)/total_students *100
overall_passing = (percentage_passing_math+percentage_passing_reading)/2

District_summary =[{"Total Schools": total_schools, "Total Students": total_students, "Total Budget": total_budget,
                  "Average Math Score": "{:.6f}".format(average_math_score), "Average Reading Score" : "{:.6f}".format(average_reading_score), 
                  "Overall Passing": overall_passing, "% Passing Math": percentage_passing_math,
                   "% Passing Reading": percentage_passing_reading, "% Overall Passing Rate": overall_passing 
                   }
                  ]

District_df =pd.DataFrame(District_summary)

#Formatting and organization
Organized_District_df= District_df[["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score",
             "Overall Passing", "% Passing Math","% Passing Reading", "% Overall Passing Rate"]]

Formatted_Organized_District_df = Organized_District_df
Formatted_Organized_District_df['Total Budget'] = Formatted_Organized_District_df['Total Budget'].map("${:,.2f}".format)

Formatted_Organized_District_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.877840,80.393158,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [69]:
school_summary_df_1= school_data.set_index('school_name')

per_student_budget = school_summary_df_1['budget']/school_summary_df_1['size']

school_groupby= school_data_complete.groupby(['school_name'])
avg_reading= school_groupby.reading_score.mean()
avg_math = school_groupby.math_score.mean()
total_students = school_groupby.student_name.count()
overall_passing = (avg_math +avg_reading)/2

passing_reading_groupby= passing_reading.groupby(['school_name']).student_name.count()
passing_math_groupby = passing_math.groupby(['school_name']).student_name.count()

percent_passing_reading = (passing_reading_groupby/total_students)*100
percent_passing_math = (passing_math_groupby/total_students)*100
percent_overall = (percent_passing_reading + percent_passing_math)/2

#merge series to create a data frame which can be merged with school_summary_df
school_summary_df_2 =pd.DataFrame({'Per Student Budget': per_student_budget, 'Average Math Score' : avg_math,
             'Average Reading Score': avg_reading, '% Passing Math': percent_passing_math,
            '% Passing Reading':percent_passing_reading, '% Overall Passing Rate': percent_overall})

school_summary_df =school_summary_df_1.join(school_summary_df_2)


#Housekeeping
del school_summary_df ['School ID']
school_summary_df = school_summary_df.rename(columns={"type":"School Type", 
                                                      "size": "Size", "budget": "School Budget"})

#formatting

formatted_school_summary_df['School Budget'] = school_summary_df['School Budget'].map("${:,.2f}".format)
formatted_school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map("${:,.2f}".format)
formatted_school_summary_df['Size'] = school_summary_df['Size'].map("{:,.0f}".format)
formatted_school_summary_df

,School Type,Size,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [70]:
top_performers= formatted_school_summary_df.sort_values('% Overall Passing Rate', ascending = False)
top_performers.head(5)

,School Type,Size,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [71]:
bottom_performers= formatted_school_summary_df.sort_values('% Overall Passing Rate')
bottom_performers.head(5)

,School Type,Size,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [72]:
grade_9 =student_data.loc[student_data['grade'] == '9th', :]
grade_10 =student_data.loc[student_data['grade'] == '10th', :]
grade_11 =student_data.loc[student_data['grade'] == '11th', :]
grade_12 =student_data.loc[student_data['grade'] == '12th', :]

grade_9_math = grade_9.groupby(['school_name']).math_score.mean()
grade_10_math = grade_10.groupby(['school_name']).math_score.mean()
grade_11_math = grade_11.groupby(['school_name']).math_score.mean()
grade_12_math = grade_12.groupby(['school_name']).math_score.mean()

math_scores_by_grade = pd.DataFrame ({"9th": grade_9_math, "10th": grade_10_math, 
                                      "11th": grade_11_math, "12th": grade_12_math
                                     }
                                    )
math_scores_by_grade.index.names = ['']
math_scores_by_grade


,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [73]:
grade_9_reading = grade_9.groupby(['school_name']).reading_score.mean()
grade_10_reading = grade_10.groupby(['school_name']).reading_score.mean()
grade_11_reading = grade_11.groupby(['school_name']).reading_score.mean()
grade_12_reading= grade_12.groupby(['school_name']).reading_score.mean()

reading_scores_by_grade = pd.DataFrame ({"9th": grade_9_reading, "10th": grade_10_reading, 
                                      "11th": grade_11_reading, "12th": grade_12_reading
                                     }
                                    )


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [74]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [75]:
school_summary_df['spending bins'] = pd.cut(school_summary_df['Per Student Budget'], spending_bins, labels = group_names)
spending_groupby = school_summary_df.groupby(['spending bins'])

math_by_spending = spending_groupby['Average Math Score' ].mean()
reading_by_spending= spending_groupby['Average Reading Score' ].mean()
math_passing_by_spending = spending_groupby['% Passing Reading'].mean()
reading_passing_by_spending= spending_groupby['% Passing Math'].mean()
passing_by_spending =spending_groupby["% Overall Passing Rate"].mean()

scores_by_school_spending = pd.DataFrame({"Average Math Score": math_by_spending , 
                                          "Average Reading Score": reading_by_spending, 
                                          "% Passing Math": math_passing_by_spending, 
                                          "% Passing Reading": reading_passing_by_spending, 
                                          "% Overall Passing Rate": passing_by_spending })

scores_by_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
spending bins,,,,,
<$585,83.455399,83.933814,96.610877,93.460096,95.035486
$585-615,83.599686,83.885211,95.900287,94.230858,95.065572
$615-645,79.079225,81.891436,86.106569,75.668212,80.887391
$645-675,76.997210,81.027843,81.133951,66.164813,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [76]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [78]:
# I am well aware that I could have created a function for these last few tasks, 
# but the repitition is better for me learning these things
school_summary_df['School Size'] = pd.cut(school_summary_df['Size'], size_bins, labels = group_names)
size_groupby = school_summary_df.groupby(['School Size'])

math_by_size = size_groupby['Average Math Score' ].mean()
reading_by_size= size_groupby['Average Reading Score' ].mean()
math_passing_by_size = size_groupby['% Passing Reading'].mean()
reading_passing_by_size= size_groupby['% Passing Math'].mean()
passing_by_size = size_groupby["% Overall Passing Rate"].mean()

scores_by_school_size = pd.DataFrame({"Average Math Score": math_by_size , 
                                      "Average Reading Score": reading_by_size, 
                                      "% Passing Math": math_passing_by_size, 
                                      "% Passing Reading": reading_passing_by_size, 
                                      "% Overall Passing Rate": passing_by_size })

scores_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,96.099437,93.550225,94.824831
Medium (1000-2000),83.374684,83.864438,96.790680,93.599695,95.195187
Large (2000-5000),77.746417,81.344493,82.766634,69.963361,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [79]:
school_type_groupby = school_summary_df.groupby(['School Type'])

math_by_type = school_type_groupby['Average Math Score' ].mean()
reading_by_type= school_type_groupby['Average Reading Score' ].mean()
math_passing_by_type = school_type_groupby['% Passing Reading'].mean()
reading_passing_by_type = school_type_groupby['% Passing Math'].mean()
passing_by_type = school_type_groupby["% Overall Passing Rate"].mean()

scores_by_school_type = pd.DataFrame({"Average Math Score": math_by_type , 
                                      "Average Reading Score": reading_by_type, 
                                      "% Passing Math": math_passing_by_type, 
                                      "% Passing Reading": reading_passing_by_type, 
                                      "% Overall Passing Rate": passing_by_type })
scores_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,96.586489,93.620830,95.103660
District,76.956733,80.966636,80.799062,66.548453,73.673757
